<a href="https://colab.research.google.com/github/IvanGroza/DSL/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
def productive_symbols(grammar) -> set:
  productive = set()
  prev_count = None

  def absent_unproductive_symbols(rule: str) -> bool:
    return all(map(lambda s: s in productive or s in grammar['toks'], rule))

  while len(productive) != prev_count:
    prev_count = len(productive)
    productive_symbols = [var for var, definition in grammar['vars'].items() if list(filter(absent_unproductive_symbols, definition)) != []]
    productive = productive.union(set(productive_symbols))

  return productive
def reachable_symbols(grammar) -> set:
  reachable = {grammar['hvar']}
  prev_count = None

  while len(reachable) != prev_count:
    prev_count = len(reachable)
    for var, definition in grammar['vars'].items():
      if var in reachable:
        for rule in definition:
          reachable = reachable.union(set(filter(lambda s: s not in grammar['toks'], rule)))

  return reachable
from copy import deepcopy

def remove_useless_symbols(grammar):
  grammar_copy = deepcopy(grammar)

  productive = productive_symbols(grammar)
  grammar_copy['vars'] = {var: [rule for rule in definition if all(map(lambda s: s in productive or s in grammar['toks'], rule))] 
                          for var, definition in grammar_copy['vars'].items()}

  reachable = reachable_symbols(grammar_copy)
  grammar_copy['vars'] = {var: [rule for rule in definition if all(map(lambda s: s in reachable or s in grammar['toks'], rule))] 
                          for var, definition in grammar_copy['vars'].items() if var in reachable}

  return grammar_copy
def nullable_symbols(grammar) -> set:
  nullable = set()
  prev_count = None

  def all_nullable_symbols(rule: str) -> bool:
    return all(map(lambda s: s in nullable or s == (0, ''), rule))

  while len(nullable) != prev_count:
    prev_count = len(nullable)
    nullable_symbols = [var for var, definition in grammar['vars'].items() 
                        if list(filter(all_nullable_symbols, definition)) != []]
    nullable = nullable.union(set(nullable_symbols))

  return nullable
def test():
  grammar = {
      'toks': {(0, ''), (1, 'a'), (2, 'b'), (3, 'c')},
      'vars': {
          'S': [[(1, 'a'), 'M'],
                [(1, 'a'), 'S'],
                ['K']], 
          'K': [[(3, 'c')], 
                [(0, '')]], 
          'M': [['M', 'P']],
          'P': [[(3, 'c')]]
          },
      'hvar': 'S'
      }

  grammar = remove_useless_symbols(grammar)
  print(grammar)

  print('Nullable symbols are ' + str(nullable_symbols(grammar)))

test()

{'toks': {(3, 'c'), (1, 'a'), (0, ''), (2, 'b')}, 'vars': {'S': [[(1, 'a'), 'S'], ['K']], 'K': [[(3, 'c')], [(0, '')]]}, 'hvar': 'S'}
Nullable symbols are {'S', 'K'}


{'toks': {(3, 'c'), (1, 'a'), (0, ''), (2, 'b')}, 'vars': {'S': [[(1, 'a'), 'S'], ['K']], 'K': [[(3, 'c')], [(0, '')]]}, 'hvar': 'S'}
Nullable symbols are {'S', 'K'}